# Check for confounding variables

This notebook uses chi-squared tests to look for clinical variables that are associated with having a chromosome event or not.

- Get clinical tables
- Get event tables
- Binarize clinical columns as needed
- For each binary column in the clinical table, make a contingency table of that column and the event table
- Run chi squared test and save results

In [1]:
import pandas as pd
import numpy as np
import os
import cptac
import altair as alt
import scipy.stats

In [2]:
pd.options.display.max_columns = None
pd.options.display.max_colwidth = None

In [3]:
dss = {
    "brca": cptac.Brca,
#     "ccrcc": cptac.Ccrcc,
    "colon": cptac.Colon,
#     "endometrial": cptac.Endometrial,
#     "gbm": cptac.Gbm,
    "hnscc": cptac.Hnscc,
    "lscc": cptac.Lscc,
    "luad": cptac.Luad,
    "ovarian": cptac.Ovarian
}

In [4]:
def load_tables(cancer_type):
    
    # Load the dataset
    ds = dss[cancer_type]()
    
    # Get the clinical table
    clin = ds.get_clinical()
    
    # Get the event table
    event = pd.\
    read_csv(f"{cancer_type}_has_event.tsv", sep="\t", index_col=0).\
    rename(columns={"gain_event": "8q_gain", "loss_event": "8p_loss"})
    
    joined = clin.join(event, how="inner")
    
    return joined

In [5]:
def test_cnv_association(df, test_cols, cnv_col):
    
    pvals = {}
    efs = {}
    
    for col in test_cols:
        
        # Create contingency table
        contingency_table = pd.crosstab(df[cnv_col], df[col])
        
        # Run test
        chi2, p, dof, exp_freq = scipy.stats.chi2_contingency(contingency_table)
        
        # Check assumptions: No group has expected value < 1, and no more than
        # 20% of groups have expected frequency < 5.
        exp_freq = pd.DataFrame(exp_freq)
        
        if (exp_freq < 1).any().any():
            pvals[col] = "Not all expected frequencies were > 1."
        elif (exp_freq < 5).sum().sum() > 0.2 * exp_freq.shape[0] * exp_freq.shape[1]:
            pvals[col] = "More than 20% of groups had expected frequency < 5."
        else:
            pvals[col] = p
            
        efs[col] = exp_freq
        
    pvals = pd.DataFrame(pvals.values(), index=pvals.keys())
    pvals = pvals.rename(columns={0: "pval"})
    
    return pvals

## BRCA

In [6]:
brca = load_tables("brca")

### Simplify the age column
For the age column, we will create groups of 15 years, and combine all > 75 years.

In [7]:
brca = brca.assign(Age=brca["Age.in.Month"] // 12)
brca = brca.assign(Age_group=(brca["Age"] // 15) * 15)
brca = brca.assign(Age_group=brca["Age_group"].where(cond=(brca["Age_group"] < 75) | (pd.isnull(brca["Age"])), other=75))

In [8]:
brca["Age_group"].value_counts().sort_index()

30.0    12
45.0    36
60.0    38
75.0    19
Name: Age_group, dtype: int64

### Simplify the stage column

We will also simplify the "Stage" column.

In [9]:
brca["Stage"].value_counts().sort_index()

Stage IA       4
Stage IIA     50
Stage IIB     20
Stage III      4
Stage IIIA    22
Stage IIIB     3
Stage IIIC     4
Name: Stage, dtype: int64

Because there are only 4 Stage I samples, we will group them with Stage II.

In [10]:
def simplify_stage(row):
    if pd.isna(row):
        return row
    elif row.startswith("Stage III"):
        return "III"
    elif row.startswith("Stage II"):
        return "I or II"
    elif row.startswith("Stage I"):
        return "I or II"
    else:
        return row
    
brca = brca.assign(Simplified_Stage=brca["Stage"].apply(simplify_stage))

In [11]:
brca["Simplified_Stage"].value_counts().sort_index()

I or II    74
III        33
Name: Simplified_Stage, dtype: int64

### Race column

There aren't enough people in the hispanic.or.latino group to satisfy the requirements of the chi-squared test. Should we drop the category, or is there a permutation testing solution?

In [12]:
brca["Race"].value_counts()

white                        78
asian                        19
black.or.african.american    14
hispanic.or.latino            4
Name: Race, dtype: int64

### Run chi-squared tests
Now we will run chi-squared tests to look for association between each variable and CNV events.

In [13]:
brca_cols = [
    "Age_group",
    "Race",
    "Simplified_Stage",
    "PAM50",
    "NMF.v2.1",
]
# Don't use gender because all female

In [14]:
test_cnv_association(
    df=brca,
    test_cols=brca_cols,
    cnv_col="8p_loss"
)

,pval
Age_group,0.245893
Race,More than 20% of groups had expected frequency < 5.
Simplified_Stage,0.973413
PAM50,0.46209
NMF.v2.1,0.837778


In [15]:
test_cnv_association(
    df=brca,
    test_cols=brca_cols,
    cnv_col="8q_gain"
)

,pval
Age_group,0.224907
Race,More than 20% of groups had expected frequency < 5.
Simplified_Stage,0.72326
PAM50,0.000236805
NMF.v2.1,0.000252255


## Colon

In [58]:
colon = load_tables("colon")

### Simplify the Age column

In [47]:
colon = colon.assign(Age_years=colon["Age"] // 12)
colon = colon.assign(Age_group=(colon["Age_years"] // 15) * 15)

In [48]:
colon["Age_group"].value_counts().sort_index()

30.0     4
45.0    31
60.0    48
75.0    19
90.0     1
Name: Age_group, dtype: int64

In [49]:
colon = colon.assign(
    Age_group=colon["Age_group"].where(cond=(colon["Age_group"] < 75) | (pd.isnull(colon["Age"])), other=75)
)
colon = colon.assign(
    Age_group=colon["Age_group"].where(cond=(colon["Age_group"] > 45) | (pd.isnull(colon["Age"])), other=30)
)

In [50]:
colon["Age_group"].value_counts().sort_index()

30.0    35
60.0    48
75.0    20
Name: Age_group, dtype: int64

### Simplify the Stage column

In [59]:
colon["Stage"].value_counts().sort_index()

Stage I      12
Stage II     42
Stage III    44
Stage IV      7
Name: Stage, dtype: int64

In [56]:
colon = colon.assign(Simplified_Stage=colon["Stage"].where(colon["Stage"] != "Stage IV", "Stage III or IV"))
colon = colon.assign(
    Simplified_Stage=colon["Simplified_Stage"].where(colon["Stage"] != "Stage III", "Stage III or IV")
)

In [63]:
colon["Simplified_Stage"].value_counts().sort_index()

Stage I            12
Stage II           42
Stage III or IV    51
Name: Simplified_Stage, dtype: int64

### Run chi-squared tests

In [24]:
colon_cols = [
    "Age_group",
    "Gender",
    "Simplified_Stage",
    "Mucinous"
]

In [25]:
test_cnv_association(
    df=colon,
    test_cols=colon_cols,
    cnv_col="8p_loss"
)

,pval
Age_group,0.111073
Gender,0.270370
Simplified_Stage,0.259718
Mucinous,0.164831


In [26]:
test_cnv_association(
    df=colon,
    test_cols=colon_cols,
    cnv_col="8q_gain"
)

,pval
Age_group,0.827627
Gender,0.765412
Simplified_Stage,0.677828
Mucinous,0.626902


## HNSCC

In [64]:
hnscc = load_tables("hnscc")

cptac warning: The HNSCC data is currently strictly reserved for CPTAC investigators. Otherwise, you are not authorized to access these data. Additionally, even after these data become publicly available, they will be subject to a publication embargo (see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details). (/home/caleb/anaconda3/envs/dev/lib/python3.7/site-packages/ipykernel_launcher.py, line 4)


### Group ages

In [65]:
hnscc = hnscc.assign(Age_group=(hnscc["age"] // 10) * 10)

In [66]:
hnscc["Age_group"].value_counts().sort_index()

20.0     1
40.0     5
50.0    37
60.0    48
70.0    14
80.0     3
Name: Age_group, dtype: int64

In [67]:
hnscc = hnscc.assign(
    Age_group=hnscc["Age_group"].where(cond=(hnscc["Age_group"] < 70) | (pd.isnull(hnscc["age"])), other=70)
)
hnscc = hnscc.assign(
    Age_group=hnscc["Age_group"].where(cond=(hnscc["Age_group"] > 40) | (pd.isnull(hnscc["age"])), other=40)
)

In [68]:
hnscc["Age_group"].value_counts().sort_index()

40.0     6
50.0    37
60.0    48
70.0    17
Name: Age_group, dtype: int64

### Simplify alcohol consumption column

We are going to combine the past drinker group with the current but less group.

Also replace the history not available group with NaN.

In [69]:
hnscc["alcohol_consum"].value_counts()

Alcohol consumption equal to or less than 2 drinks per day for men and 1 drink or less per day for women    44
Alcohol consumption history not available                                                                   23
Lifelong non-drinker                                                                                        21
Alcohol consumption more than 2 drinks per day for men and more than 1 drink per day for women              11
Consumed alcohol in the past, but currently a non-drinker                                                    3
Name: alcohol_consum, dtype: int64

In [70]:
hnscc = hnscc.replace(
    to_replace="Consumed alcohol in the past, but currently a non-drinker",
    value="Alcohol consumption equal to or less than 2 drinks per day for men and 1 drink or less per day for women"
).replace(
    to_replace="Alcohol consumption history not available",
    value=np.nan
)

# THESE NUMBERS LOOK OFF

In [71]:
hnscc["alcohol_consum"].value_counts()

Alcohol consumption equal to or less than 2 drinks per day for men and 1 drink or less per day for women    47
Lifelong non-drinker                                                                                        21
Alcohol consumption more than 2 drinks per day for men and more than 1 drink per day for women              11
Name: alcohol_consum, dtype: int64

### Run chi-squared tests

In [41]:
hnscc["alcohol_consum"].value_counts()

Alcohol consumption equal to or less than 2 drinks per day for men and 1 drink or less per day for women    44
Alcohol consumption history not available                                                                   23
Lifelong non-drinker                                                                                        21
Alcohol consumption more than 2 drinks per day for men and more than 1 drink per day for women              11
Consumed alcohol in the past, but currently a non-drinker                                                    3
Name: alcohol_consum, dtype: int64

In [42]:
hnscc_cols = [
    "Age_group",
    "alcohol_consum",
    "gender",
    "histologic_grade",
    "histologic_type",
    "patho_staging_curated",
    "smoking_history",
    "tumor_site_curated"
]

In [44]:
test_cnv_association(
    df=hnscc,
    test_cols=hnscc_cols,
    cnv_col="8p_loss"
)

,pval
Age_group,More than 20% of groups had expected frequency < 5.
alcohol_consum,Not all expected frequencies were > 1.
gender,More than 20% of groups had expected frequency < 5.
histologic_grade,0.26723
histologic_type,Not all expected frequencies were > 1.
patho_staging_curated,0.214239
smoking_history,More than 20% of groups had expected frequency < 5.
tumor_site_curated,Not all expected frequencies were > 1.


In [45]:
test_cnv_association(
    df=hnscc,
    test_cols=hnscc_cols,
    cnv_col="8q_gain"
)

,pval
Age_group,More than 20% of groups had expected frequency < 5.
alcohol_consum,More than 20% of groups had expected frequency < 5.
gender,0.484162
histologic_grade,0.855255
histologic_type,Not all expected frequencies were > 1.
patho_staging_curated,More than 20% of groups had expected frequency < 5.
smoking_history,More than 20% of groups had expected frequency < 5.
tumor_site_curated,Not all expected frequencies were > 1.


In [28]:
hnscc

,Sample_Tumor_Normal,Cored_Sample,P16,age,alcohol_consum,clinic_staging_dist_metas,country,follow_up_days,follow_up_is_contact,follow_up_vital_status,gender,histologic_grade,histologic_type,num_pack_years_sm,num_smoke_per_day,num_yrs_alc_con,patho_staging_curated,patho_staging_orignial,patho_staging_pn,patho_staging_pt,smoke_age_start,smoke_age_stop,smoking_history,smoking_inferred_binary,smoking_second_hand,tumor_focality,tumor_necrosis,tumor_site_curated,tumor_site_original,tumor_size_cm,8q_gain,8p_loss
C3L-00977,Tumor,False,Not Evaluated,56.0,Lifelong non-drinker,cM0,Russia,377.0,No,Living,Male,G1 Well differentiated,"Squamous cell carcinoma, conventional",NaN,20,NaN,Stage III,Stage III,pN1,pT1,NaN,NaN,"Current reformed smoker, years unknown",strong_evidence,Yes,Unifocal,Not identified,Oral cavity,Floor of mouth,1.2,False,False
C3L-00987,Tumor,False,Not Evaluated,61.0,"Consumed alcohol in the past, but currently a non-drinker",cM0,Ukraine,429.0,Yes,Living,Male,G2 Moderately differentiated,"Squamous cell carcinoma, conventional",43.0,20,30,Stage III,Stage III,pN1,pT2,18,NaN,Current smoker: Includes daily and non-daily smokers,strong_evidence,Yes,Unifocal,Present,Oral cavity,Tongue,4.0,True,True
C3L-00994,Tumor,False,Not Evaluated,50.0,Alcohol consumption more than 2 drinks per day for men and more than 1 drink per day for women,cM0,Ukraine,132.0,No,Deceased,Male,G2 Moderately differentiated,"Squamous cell carcinoma, conventional",10.2,6,32,Stage II,Stage II,pN0,pT2,16,50,Current reformed smoker within past 15 years,strong_evidence,Exposure to secondhand smoke history not available,Unifocal,Present,Oral cavity,Tongue,3.0,False,False
C3L-00995,Tumor,False,Not Evaluated,56.0,Alcohol consumption more than 2 drinks per day for men and more than 1 drink per day for women,cM0,Ukraine,-10.0,No,Deceased,Male,G1 Well differentiated,"Squamous cell carcinoma, conventional",31.0,20,31,Stage III,Stage III,pN1,pT2,25,56,"Current reformed smoker, more than 15 years",weak_evidence,Exposure to secondhand smoke history not available,Unifocal,Not identified,Oral cavity,Buccal mucosa,4.0,False,False
C3L-00997,Tumor,False,Not Evaluated,47.0,Lifelong non-drinker,cM0,Ukraine,442.0,Yes,Living,Male,G2 Moderately differentiated,"Squamous cell carcinoma, conventional",15.0,20,NaN,Stage II,Stage III,pN1,pT2,12,27,"Current reformed smoker, more than 15 years",strong_evidence,Yes,Unifocal,Present,Oropharynx,Oropharynx,4.0,False,True
C3L-00999,Tumor,False,Not Evaluated,56.0,Lifelong non-drinker,cM0,Russia,363.0,No,Living,Male,G1 Well differentiated,"Squamous cell carcinoma, conventional",36.0,20,NaN,Stage II,Stage II,pN0,pT2,20,NaN,Current smoker: Includes daily and non-daily smokers,weak_evidence,Yes,Unifocal,Present,Oral cavity,Floor of mouth,2.2,True,False
C3L-01138,Tumor,False,Not Evaluated,62.0,Alcohol consumption equal to or less than 2 drinks per day for men and 1 drink or less per day for women,cM0,Ukraine,363.0,No,Living,Male,G1 Well differentiated,"Squamous cell carcinoma, conventional",54.0,20,NaN,Stage IV,Stage IVA,pN1,pT4a,8,NaN,Current smoker: Includes daily and non-daily smokers,strong_evidence,Exposure to secondhand smoke history not available,Unifocal,Not identified,Larynx,Larynx,6.0,False,False
C3L-01237,Tumor,False,Not Evaluated,57.0,Alcohol consumption equal to or less than 2 drinks per day for men and 1 drink or less per day for women,cM0,Ukraine,20.0,Yes,Living,Male,G2 Moderately differentiated,"Squamous cell carcinoma, conventional",6.0,40,NaN,Stage IV,Stage IVA,pN2,pT2,17,20,"Current reformed smoker, more than 15 years",strong_evidence,Exposure to secondhand smoke history not available,Unifocal,Not identified,Oral cavity,Floor of mouth,4.0,False,True
C3L-02621,Tumor,False,Unknown,68.0,Alcohol consumption more than 2 drinks per day for men and more than 1 drink per day for women,Staging Incomplete,Bulgaria,314.0,Yes,Living,Male,G3 Poorly differentiated,"Squamous cell carcinoma, conventional",45.0,30,20,Stage I,Stage I,pNX,pT1,38,NaN,

## LSCC

In [ ]:
lscc = load_tables("lscc")

## LUAD

In [ ]:
luad = load_tables("luad")

## Ovarian

In [ ]:
ovarian = load_tables("ovarian")

Clinical vars to use
- Age
- Gender
- Race
- Tumor stage/grade
- Histology/subtype
- TP53 and other mutation status for any that have like a 10% (maybe 5%) or greater frequency

brca

- Replicate_Measurement_IDs
- Sample_Tumor_Normal
- Age.in.Month
- Gender
- Race
- Human.Readable.Label
- Experiment
- Channel
- Stage
- PAM50
- NMF.v2.1
- ER
- PR
- ER.IHC.Score
- PR.IHC.Score
- Coring.or.Excision
- Ischemia.Time.in.Minutes
- Ischemia.Group
- Necrosis
- Tumor.Cellularity
- Total.Cellularity
- In.CR
- QC.status

colon

- Sample_Tumor_Normal
- Age
- CEA
- Gender
- Lymphatic_Invasion
- Mucinous
- Perineural_Invasion
- Polyps_History
- Polyps_Present
- Stage
- Subsite
- Synchronous_Tumors
- Tumor.Status
- Vascular_Invasion
- Vital.Status
- pathalogy_N_stage
- pathalogy_T_stage

hnscc

- Sample_Tumor_Normal
- Cored_Sample
- P16
- age
- alcohol_consum
- clinic_staging_dist_metas
- country
- follow_up_days
- follow_up_is_contact
- follow_up_vital_status
- gender
- histologic_grade
- histologic_type
- num_pack_years_sm
- num_smoke_per_day
- num_yrs_alc_con
- patho_staging_curated
- patho_staging_orignial
- patho_staging_pn
- patho_staging_pt
- smoke_age_start
- smoke_age_stop
- smoking_history
- smoking_inferred_binary
- smoking_second_hand
- tumor_focality
- tumor_necrosis
- tumor_site_curated
- tumor_site_original
- tumor_size_cm

lscc

- Sample_Tumor_Normal
- Smoking.History
- Stage
- Country.of.Origin
- Age
- Gender
- Ethnicity
- Cigarettes.per.Day
- Pack.Years.Smoked
- Secondhand.Smoke

luad

- Sample.IDs
- Sample_Tumor_Normal
- Smoking.Status
- Stage
- Region.of.Origin
- Country.of.Origin
- Age
- Gender
- Ethnicity
- Height.cm
- Weight.kg
- BMI
- Cigarettes.per.Day
- Pack.Years.Smoked
- Smoking.History
- Secondhand.Smoke

ovarian

- Sample_Tumor_Normal
- Participant_Procurement_Age
- Participant_Gender
- Participant_Race
- Participant_Ethnicity
- Participant_Jewish_Heritage
- Participant_History_Malignancy
- Participant_History_Chemotherapy
- Participant_History_Neo-adjuvant_Treatment
- Participant_History_Radiation_Therapy
- Participant_History_Hormonal_Therapy
- Aliquots_Plasma
- Blood_Collection_Time
- Blood_Collection_Method
- Anesthesia_Time
- Tumor_Surgical_Devascularized_Time
- Tumor_Sample_Number
- Tumor_Sample_1_Weight
- Tumor_Sample_1_LN2_Time
- Tumor_Sample_1_Ischemia_Time
- Tumor_Sample_2_Weight
- Tumor_Sample_2_LN2_Time
- Tumor_Sample_2_Ischemia_Time
- Tumor_Sample_3_Weight
- Tumor_Sample_3_LN2_Time
- Tumor_Sample_3_Ischemia_Time
- Tumor_Sample_4_Weight
- Tumor_Sample_4_LN2_Time
- Tumor_Sample_4_Ischemia_Time
- Tumor_Sample_5_Weight
- Tumor_Sample_5_LN2_Time
- Tumor_Sample_5_Ischemia_Time
- Normal_Sample_Number
- Normal_Sample_1_Surgical_Devascularized_Time
- Normal_Sample_1_Weight
- Normal_Sample_1_LN2_Time
- Normal_Sample_1_Ischemia_Time
- Normal_Sample_2_Surgical_Devascularized_Time
- Normal_Sample_2_Weight
- Normal_Sample_2_LN2_Time
- Normal_Sample_2_Ischemia_Time
- Normal_Sample_3_Surgical_Devascularized_Time
- Normal_Sample_3_Weight
- Normal_Sample_3_LN2_Time
- Normal_Sample_3_Ischemia_Time
- Normal_Sample_4_Surgical_Devascularized_Time
- Normal_Sample_4_Weight
- Normal_Sample_4_LN2_Time
- Normal_Sample_4_Ischemia_Time
- Normal_Sample_5_Surgical_Devascularized_Time
- Normal_Sample_5_Weight
- Normal_Sample_5_LN2_Time
- Normal_Sample_5_Ischemia_Time
- Origin_Site_Disease
- Anatomic_Site_Tumor
- Anatomic_Lateral_Position_Tumor
- Histological_Subtype
- Method_of_Pathologic_Diagnosis
- Tumor_Stage_Ovary_FIGO
- Tumor_Grade
- Tumor_Residual_Disease_Max_Diameter
- Days_Between_Collection_And_Last_Contact
- Vital_Status
- Days_Between_Collection_And_Death
- Tumor_Status
- Review_Of_Initial_Pathological_Findings
- Pathology_Review_Consistent_With_Diagnosis
- Adjuvant_Radiation_Therapy
- Adjuvant_Pharmaceutical_Therapy
- Adjuvant_Immunotherapy
- Adjuvant_Hormone_Therapy
- Adjuvant_Targeted_Molecular_Therapy
- Response_After_Surgery_And_Adjuvant_Therapies
- New_Tumor_Event_After_Initial_Treatment
- New_Tumor_Event_Type
- New_Tumor_Event_Site
- Other_New_Tumor_Event_Site
- Days_Between_Collection_And_New_Tumor_Event
- New_Tumor_Event_Diagnosis
- New_Tumor_Event_Surgery
- Days_Between_Collection_And_New_Tumor_Event_Surgery
- New_Tumor_Event_Chemotherapy
- New_Tumor_Event_Immunotherapy
- New_Tumor_Event_Hormone_Therapy
- New_Tumor_Event_Targeted_Molecular_Therapy